In [1]:
import pandas as pd
import numpy as np

user_demo=pd.read_json("./User demo profiles.json")
user_demo["user_id"]=user_demo["id"]
labeled_users=pd.read_csv("./labeled_users.csv")
labeled_users["user_id"]=pd.to_numeric(labeled_users["user_id"], downcast="integer")


In [2]:
df=pd.merge(user_demo, labeled_users, left_on="user_id", right_on="user_id")
race_count=df.groupby("race").size().reset_index()
lt_21=((2021-df["year_born"])<21).reset_index().groupby("year_born").size().reset_index()
print(race_count)
print(lt_21)
print(df)

   race     0
0   1.0   328
1   2.0   203
2   3.0   116
3   4.0  2792
4   5.0   133
   year_born     0
0      False  3594
1       True    13
              id                                 name      screen_name  \
0          12488                           Chad Boyce         djsnipa1   
1         719703                             Tomato 😷           Tomato   
2         722153                                blank            blank   
3         749003  A ring made from a spoon --- E83.31            IYQbd   
4         811618                                 Mr.O      Putanginamo   
...          ...                                  ...              ...   
3602  4895390642                    Dalton's Eyeliner  FlyAwayEyeliner   
3603  4895831833                         Jamie Loftus    cowboyjamie77   
3604  4921995243                                saved         memelady   
3605  4924158634                  Winter da CoffeeCat     WinterStar21   
3606  4928657861                 magical fluf

In [41]:
import json

text_file = open("./Twitter_User_Handles_labeled_tweets.json", "r", encoding='utf-8')
data = text_file.read()
text_file.close()
tweets = json.loads(data)

In [47]:
from nltk.tokenize import word_tokenize
import numpy as np
import re 

users=list(tweets.keys())
def remove_punctuation(tweet):
    punctuation=[".",",",":",";","!","?"]
    for p in punctuation:
        tweet=tweet.replace(p, "")
    return tweet

def extract(tweet):
    tweet=remove_punctuation(tweet).lstrip().split(" ")
    tweet=list(filter(lambda x: len(x)>0, tweet))
    ats=list(filter(lambda x: x[0]=="@", tweet))
    hashtags=list(filter(lambda x: x[0]=="#",tweet))
    clean=list(filter(lambda x: x[0]!="@" and x[0]!="#" and x[0:4]!="http", tweet))
    return " ".join(clean), " ".join(ats), " ".join(hashtags)

def create_excel(users,indexes):
    users=users[indexes[0]:indexes[1]]
    df = pd.DataFrame(columns = ['user_id', 'clean_text', '@','#'])
    for user in users:
        for tweet in tweets[user]:
            clean, ats, hashtags = extract(tweet)
            df = df.append({'user_id':user, 'clean_text':clean, '@':ats,'#':hashtags},ignore_index=True)

    df.to_excel("cleaned_tweets"+str(indexes)+".xlsx")



In [48]:
indexes=[[0,1000],[1000,2000],[2000,3000],[3000,len(users)]]
create_excel(users, indexes[0])

In [49]:
create_excel(users, indexes[1])

In [50]:
create_excel(users, indexes[2])

In [51]:
create_excel(users, indexes[3])

In [52]:
df1=pd.read_excel("./cleaned_tweets[0, 1000].xlsx",engine='openpyxl')
df2=pd.read_excel("./cleaned_tweets[1000, 2000].xlsx",engine='openpyxl')
df3=pd.read_excel("./cleaned_tweets[2000, 3000].xlsx",engine='openpyxl')
df4=pd.read_excel("./cleaned_tweets[3000, 3276].xlsx",engine='openpyxl')

In [53]:
merge=df1.append(df2, ignore_index = True)
merge=merge.append(df3, ignore_index = True)
merge=merge.append(df4, ignore_index = True)


In [54]:
merge.to_excel("cleaned_tweets.xlsx")
